In [11]:
from google.colab import drive
drive.mount('/gdrive')
#before run  encoder sentences must be in descending order
loc = '/gdrive/My Drive/Colab Notebooks/seq2seq/v3/'

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [14]:

#loader1
#import class torch
import torch
#import class Dataset and DataLoader
from torch.utils.data import Dataset,DataLoader
#import numpy
import numpy as np
class load:
	def __init__(self,encoder,e_max_len,decoder,d_max_len,num_lines,batch_size):
		self.e_max_len = e_max_len
		self.d_max_len = d_max_len
		self.e_data = loader(encoder,e_max_len,num_lines)
		self.d_data = loader(decoder,d_max_len,num_lines)
	
		self.e	= DataLoader(dataset=self.e_data,batch_size=batch_size)
		self.d	= DataLoader(dataset=self.d_data,batch_size=batch_size)

class loader(Dataset):
	def __init__(self,file_name,max_length,num_lines):
		self.num_lines	= num_lines
		self.max_length	= max_length
		self.file	= open(file_name).read()
		self.vocab	= { w:i+1 for i,w in enumerate(set(self.file.split()))}
		self.sentences  = self.file.split('\n')
		self.data	= np.zeros([num_lines,max_length])
		self.seq_length	= [len(s.split()) for s in self.sentences if len(s)>0]
		i = 0
		for s in self.sentences:
			if len(s)>0:
				for j,w in enumerate(s.split()):
					self.data[i][j] = self.vocab[w]
				i += 1
		self.data	= torch.from_numpy(self.data).long()
	def __getitem__(self,index):
		return self.data[index],self.seq_length[index]
	def __len__(self):
		return self.num_lines



#training attention1
import torch
import pickle
embedding_size = 10
encoder_file = loc+"encoder"
decoder_file = loc+"decoder"
encoder_max_len = 6
decoder_max_len = 7
num_lines = 14
batch_size = 3
num_layers = 4
ld = load(encoder_file,encoder_max_len,decoder_file,decoder_max_len,num_lines,batch_size)
class Attention(torch.nn.Module):
	def __init__(self,embedding):
		super(Attention,self).__init__()
		self.embedding	= embedding
		self.emb1	= torch.nn.Embedding(len(ld.e_data.vocab)+1 , self.embedding)
		self.lstm1	= torch.nn.LSTM(num_layers=num_layers,input_size=self.embedding,hidden_size=self.embedding,batch_first=True)
		self.lin1	= torch.nn.Linear(ld.e_max_len,ld.e_max_len)
		self.soft	= torch.nn.Softmax(dim=1)

		self.emb2	= torch.nn.Embedding(len(ld.d_data.vocab)+1,self.embedding,padding_idx=0)
		self.lin2	= torch.nn.Linear(ld.e_max_len+self.embedding,self.embedding)
		self.lstm2	= torch.nn.LSTM(num_layers=num_layers,input_size=self.embedding,hidden_size=self.embedding,batch_first=True)
		self.lin3	= torch.nn.Linear(self.embedding,len(ld.d_data.vocab))
		
	def forward(self,e,d):
		loss = 0
		#print(d[0])
		dx = d[0][:,:-1]
		#print(dx)
		dy = d[0][:,1:]
		#print(dy)
		dx = dx.t()
		#print(dx)
		dy = dy.t()
		#print(dy)
		
		pe = torch.nn.utils.rnn.pack_padded_sequence(e[0],e[1],batch_first=True)
		#print(pe)
		#print(e)
		epe = self.emb1(pe[0])
		#print(epe)
		epe = torch.nn.utils.rnn.PackedSequence(epe,pe[1])
		#print(epe)
		epo,(last_hid,last_state) = self.lstm1(epe)
		
		upo = torch.nn.utils.rnn.pad_packed_sequence(epo,batch_first=True,total_length=ld.e_max_len)[0]
		#print(upo)
		#print(last_hid)
		#------------------------------------------------------------------------------------------#
		for d,l in zip(dx,dy):
			#modified hidden for multi_layer purpose+++++++++
			last_hid_last = last_hid[-1][None]
			#last_hid_last = last_hid
		
			c = (last_hid_last[-1,:,None]*upo).sum(dim=2)
			#print(c)
			c = self.lin1(c)
			#print(c)
			c = self.soft(c)
			#print(c)
			c = (c[:,:,None]*upo).sum(dim=2)
			#print(c)
			d = d[:,None]
			#print(d)
			#print(l)
			de = self.emb2(d)
			#print(de)
			c = c[:,None]
			#print(c)
			de = torch.cat([c,de],dim=2)
			#print(de)
			de = self.lin2(de)
			#print(de)
			dout,(last_hid,last_state) = self.lstm2(de,(last_hid,last_state))
			
			#print(dout)
			#print(last_hid)
			#print(dout)
			non_zero_l_pos = l.nonzero().view(-1)
			l = l[non_zero_l_pos]
			dout = dout[non_zero_l_pos]
			if len(dout)>0:
				dout = self.lin3(dout)
				#print(dout)
				dout = dout[:,-1,:]
				l    = l-1
				#print(dout)
				#print(l)
				loss += error(dout,l)
			else:
				continue
		return loss
		#no problem-----------------------------------------------------------------------------------#
		
	


attention = Attention(embedding_size)
error	  = torch.nn.CrossEntropyLoss()
optim	  = torch.optim.Adam(attention.parameters(),lr=0.01)
for epoch in range(2000):
	for e,d in zip(ld.e,ld.d):
		optim.zero_grad()
		loss = attention(e,d)
		loss.backward()
		optim.step()
		#print(loss)
print(loss)
torch.save(attention.state_dict(),loc+'attention.pt')
vocab = {"e_vocab":ld.e_data.vocab,"d_vocab":ld.d_data.vocab,"embedding":10,"e_max_len":ld.e_max_len,"d_max_len":ld.d_max_len,"num_layers":num_layers}
print(vocab)
f = open(loc+"vocab","wb")
pickle.dump(vocab,f,pickle.HIGHEST_PROTOCOL)

tensor(0.0009, grad_fn=<AddBackward0>)
{'e_vocab': {'help': 1, 'bipul': 2, 'night': 3, 'danger': 4, 'girl': 5, 'limitless': 6, 'is': 7, 'kalita': 8, 'he': 9, 'us': 10, 'life': 11, 'god': 12, 'may': 13, 'not': 14, 'time': 15, 'you': 16, 'ok': 17, 'good': 18, 'a': 19, 'for': 20, 'am': 21, 'she': 22, 'in': 23, 'dark': 24, '</s>': 25, 'i': 26, 'boy': 27, 'name': 28, 'will': 29, 'my': 30, 'oh': 31, 'come': 32}, 'd_vocab': {'xohay': 1, 'bipul': 2, 'moi': 3, 'lora': 4, 'nohoy': 5, 'ejoni': 6, 'kalita': 7, 'aahi': 8, 'karone': 9, 'he': 10, 'naam': 11, 'mor': 12, 'thik': 13, 'endhar': 14, '<s>': 15, 'jibon': 16, 'ximahin': 17, 'amar': 18, 'tai': 19, 'suwali': 20, 'bhogobane': 21, 'korok': 22, 'somoy': 23, 'ejon': 24, 'xi': 25, '</s>': 26, 'apunak': 27, 'bipangat': 28, 'ase': 29, 'bhal': 30, 'raati': 31, 'bhogoban': 32}, 'embedding': 10, 'e_max_len': 6, 'd_max_len': 7, 'num_layers': 4}


In [15]:
#result attention
import pickle
vocab = open(loc+"vocab","rb")
vocab = pickle.load(vocab)
d_vocab = {vocab["d_vocab"][i]:i for i in vocab["d_vocab"]}
num_layers = vocab["num_layers"]
import torch

def get_word(x):
	v,i=torch.max(x,0)
	return d_vocab[i.item()+1]
	
def get_tensor(x):
	return torch.LongTensor([vocab["d_vocab"][x]])

class Attention(torch.nn.Module):
	def __init__(self,embedding):
		super(Attention,self).__init__()
		self.embedding	= embedding
		self.emb1	= torch.nn.Embedding(len(vocab["e_vocab"])+1 , self.embedding)
		self.lstm1	= torch.nn.LSTM(num_layers = num_layers,input_size=self.embedding,hidden_size=self.embedding,batch_first=True)
		self.lin1	= torch.nn.Linear(vocab["e_max_len"],vocab["e_max_len"])
		self.soft	= torch.nn.Softmax(dim=1)

		self.emb2	= torch.nn.Embedding(len(vocab["d_vocab"])+1,self.embedding,padding_idx=0)
		self.lin2	= torch.nn.Linear(vocab["e_max_len"]+self.embedding,self.embedding)
		self.lstm2	= torch.nn.LSTM(num_layers = num_layers,input_size=self.embedding,hidden_size=self.embedding,batch_first=True)
		self.lin3	= torch.nn.Linear(self.embedding,len(vocab["d_vocab"]))
		
	def forward(self,e,d):
		pe=torch.nn.utils.rnn.pack_padded_sequence(e,[len(e[0])],batch_first=True)
		e = self.emb1(pe[0])
		#print(e)
		pe=torch.nn.utils.rnn.PackedSequence(e,pe[1])
		epo,(last_hid,last_state) = self.lstm1(pe)
		
		upo = torch.nn.utils.rnn.pad_packed_sequence(epo,batch_first=True,total_length=vocab["e_max_len"])[0]
		#print(upo)
		#print(last_hid)
		word = ""
		#++
		#d = d[:,None]
		#++
		#de = self.emb2(d)
		while word!="</s>":
		#------------------------------------------------------------------------------------------#
			#modified hidden for multi_layer purpose+++++++++
			last_hid_last = last_hid[-1][None]
			#last_hid_last = last_hid
			
			c = (last_hid_last[-1,:,None]*upo).sum(dim=2)
			c = self.lin1(c)
			#print(c)
			c = self.soft(c)
			#print(c)
			c = (c[:,:,None]*upo).sum(dim=2)
			#print(c)
			#--
			d = d[:,None]
			#print(d)
			#print(l)
			#--
			de = self.emb2(d)
			#print(de)
			c = c[:,None]
			de = torch.cat([c,de],dim=2)
			#print(de)
			de = self.lin2(de)
			#print(de)
			dout,(last_hid,last_state) = self.lstm2(de,(last_hid,last_state))
			#print(dout)
			#print(last_hid)
			#break
			if len(dout)>0:
				dout = self.lin3(dout)
				dout = dout[:,-1,:]
				word = get_word(dout[0])
				print(word)
				d    = get_tensor(word)
				#++d     = last_hid
		return	
		#no problem-----------------------------------------------------------------------------------#
		
	


model = Attention(vocab["embedding"])
model.load_state_dict(torch.load(loc+'attention.pt'),strict=False)
for i in vocab['e_vocab']:
	if(i!='</s>'):
		print(i)
		print('__________')
		sentence = i+" </s>"
		indexing = torch.LongTensor([[vocab["e_vocab"][s] for s in sentence.split()]])
		d = model(indexing,get_tensor("<s>"))
		print('***************')

help
__________
endhar
raati
</s>
***************
bipul
__________
mor
xohay
</s>
***************
night
__________
endhar
</s>
***************
danger
__________
somoy
raati
</s>
***************
girl
__________
mor
xohay
</s>
***************
limitless
__________
somoy
raati
</s>
***************
is
__________
thik
</s>
***************
kalita
__________
somoy
kalita
</s>
***************
he
__________
mor
xohay
</s>
***************
us
__________
endhar
raati
</s>
***************
life
__________
jibon
</s>
***************
god
__________
bhogoban
raati
</s>
***************
may
__________
mor
bipangat
</s>
***************
not
__________
thik
ase
</s>
***************
time
__________
ximahin
</s>
***************
you
__________
somoy
raati
</s>
***************
ok
__________
thik
ase
</s>
***************
good
__________
mor
xohay
</s>
***************
a
__________
he
bhal
</s>
***************
for
__________
mor
xohay
</s>
***************
am
__________
somoy
kalita
</s>
***************
she
________

In [ ]:
for i in vocab['d_vocab']:
  print(i)